In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

from histopatseg.evaluation.utils import aggregate_tile_embeddings

In [ ]:
project_dir = Path(".").resolve().parent
print(f"Project Directory: {project_dir}")

In [ ]:
embedding_file = project_dir / "data/processed/embeddings/lunghist700_20x_UNI2_embeddings.npz"
metadata  = pd.read_csv(project_dir / "/home/valentin/workspaces/histopatseg/data/processed/LungHist700_tiled/LungHist700_20x/metadata.csv").set_index("tile_id")
metadata.head()

In [ ]:
data = np.load(embedding_file)
embeddings = data["embeddings"]
tile_ids = data["tile_ids"]
embedding_dim = data["embedding_dim"]

# Print basic information
print(f"Loaded {len(embeddings)} embeddings with dimensionality {embeddings.shape[1]}")
print(f"Embedding dimension from model: {embedding_dim}")

In [ ]:
# Check if all embedding tile_ids are in the metadata index
missing_ids = [id for id in tile_ids if id not in metadata.index]
if missing_ids:
    print(f"Warning: {len(missing_ids)} tile_ids from embeddings are not in metadata")
    print(f"First few missing IDs: {missing_ids[:5]}")
aligned_metadata = metadata.reindex(tile_ids)
aligned_metadata['subclass'] = aligned_metadata.apply(
    lambda row: row['superclass'] if pd.isna(row['subclass']) and row['superclass'] == 'nor' else row['subclass'], 
    axis=1
)

In [ ]:
aggregated_embeddings, aggregated_metadata = aggregate_tile_embeddings(
    embeddings=embeddings,
    tile_ids=tile_ids,
    metadata=aligned_metadata,
    group_by="original_filename",
)